##  To do:
- hh; jobs (office, industrial, retail); housing units (sf, mf)
    - 2019 (average of runs)
    - 2050 (average of runs)

- append to parcel eq table
    - create some pivots

In [12]:
import os
import pandas as pd
import glob
import numpy as np

# show all columns
pd.options.display.max_columns = None

In [13]:
outputs = os.path.join(r'.\Outputs')
if not os.path.exists(outputs):
    os.makedirs(outputs)

In [14]:
# Parcel Equivalency Table
eq = pd.read_csv(r".\Inputs\parcel_eq_v5.csv")

# get 2019 and 2050 files
remm_pm_2019_1 = pd.read_csv(r"D:\Josh_Projects\REMM2_For_Python3_Internal_Use_1\REMMRun\Progression_Metrics\run_48_year_2019_parcel_progression_metrics.csv")
remm_pm_2050_1 = pd.read_csv(r"D:\Josh_Projects\REMM2_For_Python3_Internal_Use_1\REMMRun\Progression_Metrics\run_48_year_2050_parcel_progression_metrics.csv")

remm_pm_2019_2 = pd.read_csv(r"D:\Josh_Projects\REMM2_For_Python3_Internal_Use_2\REMMRun\Progression_Metrics\run_48_year_2019_parcel_progression_metrics.csv")
remm_pm_2050_2 = pd.read_csv(r"D:\Josh_Projects\REMM2_For_Python3_Internal_Use_2\REMMRun\Progression_Metrics\run_48_year_2050_parcel_progression_metrics.csv")

remm_pm_2019_3 = pd.read_csv(r"D:\Josh_Projects\REMM2_For_Python3_Internal_Use_3\REMMRun\Progression_Metrics\run_48_year_2019_parcel_progression_metrics.csv")
remm_pm_2050_3 = pd.read_csv(r"D:\Josh_Projects\REMM2_For_Python3_Internal_Use_3\REMMRun\Progression_Metrics\run_48_year_2050_parcel_progression_metrics.csv")

remm_pm_2019_4 = pd.read_csv(r"D:\Josh_Projects\REMM2_For_Python3_Internal_Use_4\REMMRun\Progression_Metrics\run_47_year_2019_parcel_progression_metrics.csv")
remm_pm_2050_4 = pd.read_csv(r"D:\Josh_Projects\REMM2_For_Python3_Internal_Use_4\REMMRun\Progression_Metrics\run_47_year_2050_parcel_progression_metrics.csv")

remm_pm_2019_5 = pd.read_csv(r"D:\Josh_Projects\REMM2_For_Python3_Internal_Use_5\REMMRun\Progression_Metrics\run_47_year_2019_parcel_progression_metrics.csv")
remm_pm_2050_5 = pd.read_csv(r"D:\Josh_Projects\REMM2_For_Python3_Internal_Use_5\REMMRun\Progression_Metrics\run_47_year_2050_parcel_progression_metrics.csv")

remm_pm_2019_6 = pd.read_csv(r"D:\Josh_Projects\REMM2_For_Python3_Internal_Use_6\REMMRun\Progression_Metrics\run_46_year_2019_parcel_progression_metrics.csv")
remm_pm_2050_6 = pd.read_csv(r"D:\Josh_Projects\REMM2_For_Python3_Internal_Use_6\REMMRun\Progression_Metrics\run_46_year_2050_parcel_progression_metrics.csv")

In [15]:
def prepare_df(df, year):
    df = df.set_index('parcel_id')
    df.loc[(df['is_sf']==1), 'sf_units'] = df['residential_units']
    df.loc[(df['is_mf']==1), 'mf_units'] = df['residential_units']
    df['ind_jobs'] = df['jobs_wholesale'] + df['jobs_manuf']
    df['rtl_jobs'] = df['jobs_retail'] + df['jobs_accom_food']
    df['off_jobs'] = df['jobs_office'] + df['jobs_gov_edu'] + df['jobs_health'] + df['jobs_other']
    df.loc[(df['has_buildings'] != 1), 'vacant_acres'] = df['parcel_acres']
    df.loc[(df['has_buildings'] != 1) & (df['developable'] == 1), 'vacant_devacres'] = df['parcel_acres']
    df['vacant_acres'].fillna(0, inplace=True)
    df['vacant_devacres'].fillna(0, inplace=True)

    df = df[['sf_units', 'mf_units', 'job_spaces', 'ind_jobs', 'rtl_jobs', 'off_jobs', 'vacant_acres', 'vacant_devacres']].copy()
    new_col_names = [col + f'_{str(year)[2:]}' for col in list(df.columns)]
    df.columns = new_col_names
    return df.fillna(0)

In [16]:
remm_pm_2019 = [remm_pm_2019_1, remm_pm_2019_2, remm_pm_2019_3, remm_pm_2019_4, remm_pm_2019_5, remm_pm_2019_6]
remm_pm_2050 = [remm_pm_2050_1, remm_pm_2050_2, remm_pm_2050_3, remm_pm_2050_4, remm_pm_2050_5, remm_pm_2050_6]

remm_pm_2019_processed = [prepare_df(df, 2019) for df in remm_pm_2019]
remm_pm_2050_processed = [prepare_df(df, 2050) for df in remm_pm_2050]

In [17]:
# stack all 6 runs together
data_stack_2019 = pd.concat(remm_pm_2019_processed)
data_stack_2050 = pd.concat(remm_pm_2050_processed)

# average the 6 runs
results_2019 = data_stack_2019.groupby(data_stack_2019.index).mean().reset_index().round().astype(int)
results_2050 = data_stack_2050.groupby(data_stack_2050.index).mean().reset_index().round().astype(int)

results_2019['residential_units_19'] = results_2019['sf_units_19'] + results_2019['mf_units_19']
results_2019['ttl_jobs_19'] = results_2019['off_jobs_19'] + results_2019['rtl_jobs_19'] + results_2019['ind_jobs_19']
results_2050['residential_units_50'] = results_2050['sf_units_50'] + results_2050['mf_units_50']
results_2050['ttl_jobs_50'] = results_2050['off_jobs_50'] + results_2050['rtl_jobs_50'] + results_2050['ind_jobs_50']


In [18]:
# fill na's in parcel eq
eq.fillna({'CENTER_NAME':'Non-Center', 'CENTER_TYPE':'Non-Center', 'CITY_AREA':'Non-City-Area'}, inplace=True)

In [19]:
# export
results = eq.merge(results_2019, on='parcel_id', how='left').merge(results_2050, on='parcel_id', how='left')
results.to_csv(os.path.join(outputs, 'SE_by_PARCEL.csv'), index = False)

In [20]:
# eq.columns

In [21]:
results.columns

Index(['parcel_id', 'TAZID_832', 'TAZID_900', 'CENTER_NAME', 'CENTER_TYPE',
       'CITY_AREA', 'COUNTY_NAME', 'COUNTY_FIPS', 'MPO', 'INFILL1990',
       'parcel_acres', 'sf_units_19', 'mf_units_19', 'job_spaces_19',
       'ind_jobs_19', 'rtl_jobs_19', 'off_jobs_19', 'vacant_acres_19',
       'vacant_devacres_19', 'residential_units_19', 'ttl_jobs_19',
       'sf_units_50', 'mf_units_50', 'job_spaces_50', 'ind_jobs_50',
       'rtl_jobs_50', 'off_jobs_50', 'vacant_acres_50', 'vacant_devacres_50',
       'residential_units_50', 'ttl_jobs_50'],
      dtype='object')

In [22]:
# aggregate to different geographies
data_columns = ['parcel_acres', 'residential_units_19', 'sf_units_19', 'mf_units_19', 
                'job_spaces_19', 'ind_jobs_19', 'rtl_jobs_19', 'off_jobs_19', 'ttl_jobs_19', 'vacant_acres_19', 'vacant_devacres_19',
                'residential_units_50', 'sf_units_50', 'mf_units_50', 'job_spaces_50',
                'ind_jobs_50', 'rtl_jobs_50', 'off_jobs_50','ttl_jobs_50', 'vacant_acres_50','vacant_devacres_50']

# aggregations
results.groupby('CITY_AREA')[data_columns].sum().reset_index().to_csv(os.path.join(outputs, 'SE_by_CITY_AREA.csv'), index = False)
results.groupby('COUNTY_NAME')[data_columns].sum().reset_index().to_csv(os.path.join(outputs, 'SE_by_COUNTY.csv'), index = False)
results.groupby('CENTER_NAME')[data_columns].sum().reset_index().to_csv(os.path.join(outputs, 'SE_by_CENTER_NAME.csv'), index = False)
results.groupby('CENTER_TYPE')[data_columns].sum().reset_index().to_csv(os.path.join(outputs, 'SE_by_CENTER_TYPE.csv'), index = False)
results.groupby('INFILL1990')[data_columns].sum().reset_index().to_csv(os.path.join(outputs, 'SE_by_INFILL1990.csv'), index = False)
results.groupby('MPO')[data_columns].sum().reset_index().to_csv(os.path.join(outputs, 'SE_by_MPO.csv'), index = False)

# pivots
results.pivot_table(index='CENTER_TYPE', columns='COUNTY_NAME', values=data_columns,  aggfunc='sum').fillna(0).reset_index().to_csv(os.path.join(outputs, 'SE_by_CENTER_TYPE_by_COUNTY.csv'), index = False)
results.pivot_table(index='COUNTY_NAME', columns='INFILL1990', values=data_columns,  aggfunc='sum').fillna(0).reset_index().to_csv(os.path.join(outputs, 'SE_by_INFILL1990_by_COUNTY.csv'), index = False)

In [ ]:
# taz 9.0 summary for MB
results['res_units_per_acre_19'] = round(results['residential_units_19'] / results['parcel_acres'], 2)
results['sf_units_per_acre_19'] = round(results['sf_units_19'] / results['parcel_acres'], 2)
results['mf_units_per_acre_19'] = round(results['mf_units_19'] / results['parcel_acres'], 2)
results['job_spaces_per_acre_19'] = round(results['job_spaces_19'] / results['parcel_acres'], 2)
results['ttl_jobs_per_acre_19'] = round(results['ttl_jobs_19'] / results['parcel_acres'], 2)
results['ind_jobs_per_acre_19'] = round(results['ind_jobs_19'] / results['parcel_acres'], 2)
results['rtl_jobs_per_acre_19'] = round(results['rtl_jobs_19'] / results['parcel_acres'], 2)
results['off_jobs_per_acre_19'] = round(results['off_jobs_19'] / results['parcel_acres'], 2)

results['res_units_per_acre_50'] = round(results['residential_units_50'] / results['parcel_acres'], 2)
results['sf_units_per_acre_50'] = round(results['sf_units_50'] / results['parcel_acres'], 2)
results['mf_units_per_acre_50'] = round(results['mf_units_50'] / results['parcel_acres'], 2)
results['job_spaces_per_acre_50'] = round(results['job_spaces_50'] / results['parcel_acres'], 2)
results['ttl_jobs_per_acre_50'] = round(results['ttl_jobs_50'] / results['parcel_acres'], 2)
results['ind_jobs_per_acre_50'] = round(results['ind_jobs_50'] / results['parcel_acres'], 2)
results['rtl_jobs_per_acre_50'] = round(results['rtl_jobs_50'] / results['parcel_acres'], 2)
results['off_jobs_per_acre_50'] = round(results['off_jobs_50'] / results['parcel_acres'], 2)

density_columns = ['res_units_per_acre_19', 'res_units_per_acre_50',
                   'sf_units_per_acre_19', 'sf_units_per_acre_50',
                   'mf_units_per_acre_19', 'mf_units_per_acre_50',
                   'job_spaces_per_acre_19', 'job_spaces_per_acre_50', 
                   'ttl_jobs_per_acre_19', 'ttl_jobs_per_acre_50', 
                   'ind_jobs_per_acre_19', 'ind_jobs_per_acre_50', 
                   'rtl_jobs_per_acre_19', 'rtl_jobs_per_acre_50', 
                   'off_jobs_per_acre_19', 'off_jobs_per_acre_50']

# density using aggregated values
taz_sum = results.groupby('TAZID_900')[data_columns].sum().reset_index()
taz_sum['res_units_per_acre_19'] = round(taz_sum['residential_units_19'] / taz_sum['parcel_acres'], 2)
taz_sum['sf_units_per_acre_19'] = round(taz_sum['sf_units_19'] / taz_sum['parcel_acres'], 2)
taz_sum['mf_units_per_acre_19'] = round(taz_sum['mf_units_19'] / taz_sum['parcel_acres'], 2)
taz_sum['job_spaces_per_acre_19'] = round(taz_sum['job_spaces_19'] / taz_sum['parcel_acres'], 2)
taz_sum['ttl_jobs_per_acre_19'] = round(taz_sum['ttl_jobs_19'] / taz_sum['parcel_acres'], 2)
taz_sum['ind_jobs_per_acre_19'] = round(taz_sum['ind_jobs_19'] / taz_sum['parcel_acres'], 2)
taz_sum['rtl_jobs_per_acre_19'] = round(taz_sum['rtl_jobs_19'] / taz_sum['parcel_acres'], 2)
taz_sum['off_jobs_per_acre_19'] = round(taz_sum['off_jobs_19'] / taz_sum['parcel_acres'], 2)

taz_sum['res_units_per_acre_50'] = round(taz_sum['residential_units_50'] / taz_sum['parcel_acres'], 2)
taz_sum['sf_units_per_acre_50'] = round(taz_sum['sf_units_50'] / taz_sum['parcel_acres'], 2)
taz_sum['mf_units_per_acre_50'] = round(taz_sum['mf_units_50'] / taz_sum['parcel_acres'], 2)
taz_sum['job_spaces_per_acre_50'] = round(taz_sum['job_spaces_50'] / taz_sum['parcel_acres'], 2)
taz_sum['ttl_jobs_per_acre_50'] = round(taz_sum['ttl_jobs_50'] / taz_sum['parcel_acres'], 2)
taz_sum['ind_jobs_per_acre_50'] = round(taz_sum['ind_jobs_50'] / taz_sum['parcel_acres'], 2)
taz_sum['rtl_jobs_per_acre_50'] = round(taz_sum['rtl_jobs_50'] / taz_sum['parcel_acres'], 2)
taz_sum['off_jobs_per_acre_50'] = round(taz_sum['off_jobs_50'] / taz_sum['parcel_acres'], 2)

taz_sum = taz_sum[['TAZID_900','parcel_acres', 'vacant_acres_19', 'vacant_acres_50', 'vacant_devacres_19', 'vacant_devacres_50'] + density_columns].copy()
taz_sum.set_index('TAZID_900', inplace=True)

# max and mean densities of parcels within taz
taz_max = results.groupby('TAZID_900')[density_columns].max()
max_col_names = ['max_' + col for col in list(taz_max.columns)]
taz_max.columns = max_col_names

taz_mean = round(results.groupby('TAZID_900')[density_columns].mean(), 2)
mean_col_names = ['mean_' + col for col in list(taz_mean.columns)]
taz_mean.columns = mean_col_names

# export
taz = taz_sum.merge(taz_max, left_index=True, right_index=True, how='left').merge(taz_mean, left_index=True, right_index=True, how='left')
taz.replace([np.inf, -np.inf, np.nan], 0, inplace=True)
taz.to_csv(os.path.join(outputs, 'SE_Density_TAZ900.csv'))